# Бинарная классификация мошеннических транзакций
## Часть 2. Инжиниринг данных и отбор признаков

### План

1. [Исходные данные]
2. [Разделение на train/test]
3. [Feature Engineering на train]
4. [Отбор признаков на train]
5. [Ребалансировка **только** train]
6. [Обучение модели]
7. [Оценка на **оригинальном** test]






In [84]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [85]:
df_train_scaled = pd.read_csv("train_scaled.csv")
df_original_test = pd.read_csv("test_scaled.csv")

### Train Test Split

In [86]:
X = df_train_scaled.drop(columns=['Class'])
y = df_train_scaled['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # стратификация!
)

### Инжиниринг новых признаков на тренировочных данных

In [87]:
# Взаимодействия и полиномы
X_train['V6_V27'] = X_train['V6'] * X_train['V27']
X_train['Amount_sq'] = X_train['Amount'] ** 2

# Бинарные маркеры выбросов (3σ)
for col in ['V6', 'Amount']:
    X_train[f'IsOutlier_{col}'] = (np.abs(X_train[col]) > 3).astype(int)

# Добавим
train_std = {'V6': X_train['V6'].std(), 'Amount': X_train['Amount'].std()}

# Логарифмирование суммы
X_train['Log_Amount'] = np.log1p(X_train['Amount'])

# Кластеризация K-means
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_train[['V6', 'V20', 'V23', 'V27']])
X_train['Cluster'] = kmeans.predict(X_train[['V6', 'V20', 'V23', 'V27']])

### Инжиниринг новых признаков на валидационных данных

In [88]:
X_test['V6_V27'] = X_test['V6'] * X_test['V27']
X_test['Amount_sq'] = X_test['Amount'] ** 2

# Бинарные маркеры выбросов. Обратить внимание на порог `> 3 * train_std[col]`
for col in ['V6', 'Amount']:
    X_test[f'IsOutlier_{col}'] = (np.abs(X_test[col]) > 3 * train_std[col]).astype(int)

# Логарифмирование суммы
X_test['Log_Amount'] = np.log1p(X_test['Amount'])

# Кластеризация (используем обученный кластеризатор!)
X_test['Cluster'] = kmeans.predict(X_test[['V6', 'V20', 'V23', 'V27']])

### Инжиниринг новых признаков на тестовых данных

In [89]:
df_original_test['V6_V27'] = df_original_test['V6'] * df_original_test['V27']
df_original_test['Amount_sq'] = df_original_test['Amount'] ** 2

for col in ['V6', 'Amount']:
    df_original_test[f'IsOutlier_{col}'] = (np.abs(df_original_test[col]) > 3 * train_std[col]).astype(int)

df_original_test['Log_Amount'] = np.log1p(df_original_test['Amount'])
df_original_test['Cluster'] = kmeans.predict(df_original_test[['V6', 'V20', 'V23', 'V27']])

**Итог**: взаимодействия (`V6_V27`), полиномы (`Amount_sq`), бинарные маркеры выбросов (`IsOutlier_V6`), логарифмированная сумма (`Log_Amount`) и кластеры (`Cluster`) — созданы для выявления нелинейных зависимостей и аномалий.

### Отбор признаков

In [90]:
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [91]:
# LightGBM Feature Importance
lgb = LGBMClassifier(random_state=42, class_weight='balanced')
lgb.fit(X_train, y_train)
importance = pd.Series(lgb.feature_importances_, index=X_train.columns)
selected_features_imp = importance.nlargest(15).index.tolist()

[LightGBM] [Info] Number of positive: 375, number of negative: 174928
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8424
[LightGBM] [Info] Number of data points in the train set: 175303, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


In [92]:
# L1-регуляризация
# selector = SelectFromModel(
#     estimator=LogisticRegression(
#         penalty='l1',
#         solver='saga',
#         class_weight='balanced',
#         max_iter=500,
#         random_state=42
#     ),
#     threshold="median"
# )



In [93]:
# selector.fit(X_train, y_train)
# selected_features_l1 = X.columns[selector.get_support()]

In [94]:
# print("Выбрано признаков:", sum(selector.get_support()))

**Методы отбора**:
*Feature Importance* (LightGBM) и [~L1-регуляризация~] — выбрали признаки с наибольшим влиянием на целевую переменную и устранили мультиколлинеарность.

In [95]:
# Итоговый выбор / объединение признаков
final_features = list(
    set(selected_features_imp) |
    # set(selected_features_l1) |
    {'V6_V27', 'Cluster', 'IsOutlier_V6'}
)
final_features

['V6_V27',
 'Cluster',
 'V8',
 'V21',
 'V22',
 'V12',
 'V9',
 'V16',
 'V14',
 'V7',
 'Amount_sq',
 'IsOutlier_V6',
 'V4',
 'V20',
 'V18',
 'V6',
 'V3',
 'V26']

**В итоговый датасет вошли**: `V6_V27`, `Cluster`, `IsOutlier_V6`, а также ключевые исходные признаки ( `V3`, `V14` и др.), как наиболее значимые для разделения классов

In [96]:
import json

missing_in_train = set(final_features) - set(X_train.columns)
missing_in_test = set(final_features) - set(X_test.columns)

if missing_in_train:
    raise ValueError(f"Признаки отсутствуют в трейне: {missing_in_train}")
if missing_in_test:
    raise ValueError(f"Признаки отсутствуют в тесте: {missing_in_test}")

X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

# Внесение изменений в оригинальный test dataset
df_original_test = df_original_test[final_features]
df_original_test.to_csv("test_feat_select.csv", index=False)

In [97]:
# Сохранение датасета с отобранными признаками
train_df = pd.concat([X_train_final, y_train], axis=1)
test_df = pd.concat([X_test_final, y_test], axis=1)
final_df = pd.concat([train_df, test_df], axis=0)
final_df.to_csv("train_feat_select.csv", index=False)

# Сохранение списка признаков
with open('selected_features.json', 'w') as f:
    json.dump(list(final_features), f)

In [98]:
del X_train_final, X_test_final, df_train_scaled, df_test_scaled, train_df, test_df, final_df

## Логистическая регрессия

In [99]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [100]:
df = pd.read_csv("train_feat_select.csv")
X = df.drop(columns=['Class'])
y = df['Class']

In [101]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # стратификация для баланса классов
)

### Ребалансировка классов

In [102]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.1, random_state=42)  # Доводим до 10% меньшего класса
under = RandomUnderSampler(sampling_strategy=0.5)     # Затем уменьшаем больший класс до соотношения 1:2

pipeline = Pipeline([
    ('o', over),
    ('u', under)
])

X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)

In [103]:
lr_model = LogisticRegression(
    solver="saga",
    class_weight="balanced",
    tol=1e-2,
    max_iter=500,
    random_state=42
)

param_grid = {
    "penalty": ["l1", "l2"],
    "C": [0.01, 0.1, 1.0],  # Сила регуляризации
    "solver": ["liblinear", "saga"]  # Для L1/L2
}

grid_search = GridSearchCV(
    estimator=lr_model,
    param_grid=param_grid,
    cv=5,
    scoring="recall",
    n_jobs=-1
)

In [104]:
# Обучение на тренировочных данных
grid_search.fit(X_train_resampled, y_train_resampled)

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced', max_iter=500,
                                          random_state=42, solver='saga',
                                          tol=0.01),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1.0], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             scoring='recall')

In [105]:
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая recall:", grid_search.best_score_)

Лучшие параметры: {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}
Лучшая recall: 0.7758401771575347


### Отчёт

In [106]:
from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.68      0.81     43732
           1       0.01      0.76      0.01        94

    accuracy                           0.68     43826
   macro avg       0.50      0.72      0.41     43826
weighted avg       1.00      0.68      0.81     43826



### Итог:

Произведены следующие этапы:

1. Разделение на train/test
2. Feature Engineering
3. Отбор признаков с помощью LightGBM
4. Ребалансировка классов в связи с сильным диcбалансом
5. Обучение модели логистической регрессии с подбором гиперпараметров через GridSearch, используя scoring="recall"

